In [16]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('list.json').read()
intents = json.loads(data_file)


for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)


pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")


# Create model - 3 layers. First layer  neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(256, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")


41 documents
9 classes ['book_avail', 'due', 'eresource', 'goodbye', 'greeting', 'issue_time', 'options', 'thanks', 'time']
79 unique lemmatized words ["'s", ',', 'about', 'anyone', 'are', 'available', 'awesome', 'be', 'book', 'bye', 'can', 'charge', 'chatting', 'close', 'closed', 'could', 'currently', 'day', 'do', 'downloadable', 'e-resource', 'fine', 'for', 'from', 'get', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'hola', 'how', 'i', 'in', 'is', 'issue', 'issuing', 'last', 'late', 'later', 'let', 'library', 'material', 'me', 'next', 'nice', 'not', 'of', 'offered', 'on', 'open', 'overdue', 'please', 'provide', 'see', 'show', 'some', 'submission', 'submitted', 'support', 'take', 'tell', 'thank', 'thanks', 'that', 'the', 'there', 'till', 'time', 'timing', 'to', 'we', 'what', 'when', 'you']
Training data created
Epoch 1/200
41/41 [==============================] - 0s 2ms/step - loss: 2.1887 - accuracy: 0.1951
Epoch 2/200
41/41 [========================

41/41 [==============================] - 0s 291us/step - loss: 0.0364 - accuracy: 1.0000
Epoch 74/200
41/41 [==============================] - 0s 293us/step - loss: 0.0508 - accuracy: 0.9756
Epoch 75/200
41/41 [==============================] - 0s 293us/step - loss: 0.0150 - accuracy: 1.0000
Epoch 76/200
41/41 [==============================] - 0s 292us/step - loss: 0.0150 - accuracy: 1.0000
Epoch 77/200
41/41 [==============================] - 0s 391us/step - loss: 0.0189 - accuracy: 1.0000
Epoch 78/200
41/41 [==============================] - 0s 389us/step - loss: 0.0425 - accuracy: 1.0000
Epoch 79/200
41/41 [==============================] - 0s 293us/step - loss: 0.0268 - accuracy: 1.0000
Epoch 80/200
41/41 [==============================] - 0s 293us/step - loss: 0.0098 - accuracy: 1.0000
Epoch 81/200
41/41 [==============================] - 0s 391us/step - loss: 0.0203 - accuracy: 1.0000
Epoch 82/200
41/41 [==============================] - 0s 292us/step - loss: 0.0129 - accuracy: 

41/41 [==============================] - 0s 293us/step - loss: 0.0174 - accuracy: 1.0000
Epoch 153/200
41/41 [==============================] - 0s 293us/step - loss: 0.0083 - accuracy: 1.0000
Epoch 154/200
41/41 [==============================] - 0s 294us/step - loss: 0.0060 - accuracy: 1.0000
Epoch 155/200
41/41 [==============================] - 0s 292us/step - loss: 0.0201 - accuracy: 1.0000
Epoch 156/200
41/41 [==============================] - 0s 293us/step - loss: 0.0083 - accuracy: 1.0000
Epoch 157/200
41/41 [==============================] - 0s 293us/step - loss: 0.0097 - accuracy: 1.0000
Epoch 158/200
41/41 [==============================] - 0s 293us/step - loss: 0.0135 - accuracy: 1.0000
Epoch 159/200
41/41 [==============================] - 0s 292us/step - loss: 0.0025 - accuracy: 1.0000
Epoch 160/200
41/41 [==============================] - 0s 293us/step - loss: 0.0198 - accuracy: 1.0000
Epoch 161/200
41/41 [==============================] - 0s 195us/step - loss: 0.0095 - a